### **Setup**

In [1]:
%%bash
pip install imbalanced-learn -q

### **Library Imports**

In [2]:
import os
import pickle
import imblearn
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

### **Configuration**

In [3]:
class CFG(object):
    def __init__(self,
                 seed: int = 42,
                 n_splits: int = 5,
                 show_info: bool = False,
                 ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.train_data_read_path = "../input/tabular-playground-series-aug-2022/train.csv"
        self.test_data_read_path = "../input/tabular-playground-series-aug-2022/test.csv"
        self.ss_data_read_path = "../input/tabular-playground-series-aug-2022/sample_submission.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)
si_mean = SimpleImputer(missing_values=np.nan, strategy="mean")

### **Helpers**

In [4]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")
    

def print_scores(accuracy: float, auc: float, precision: np.ndarray, recall: np.ndarray, f_score: np.ndarray) -> None:
    print(f"Accuracy  : {accuracy:.5f}")
    print(f"ROC-AUC   : {auc:.5f}")
    print(f"Precision : {precision}")
    print(f"Recall    : {recall}")
    print(f"F-Score   : {f_score}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    accuracy = accuracy_score(y_pred, y_true)
    auc = roc_auc_score(y_pred, y_true)
    precision, recall, f_score, _ = precision_recall_fscore_support(y_pred, y_true)

    return accuracy, auc, precision, recall, f_score

### **Model**

In [5]:
class Model(object):
    def __init__(self, model_name: str, preprocessor, seed: int):
        self.model_name = model_name

        if self.model_name == "lgr":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", LogisticRegression(random_state=seed)),
                ]
            )
        
        elif self.model_name == "knc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", KNeighborsClassifier()),
                ]
            )

        
        elif self.model_name == "dtc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", DecisionTreeClassifier(random_state=seed)),
                ]
            )

        elif self.model_name == "etc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreeClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "rfc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", RandomForestClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GradientBoostingClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "abc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", AdaBoostClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "etcs":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", ExtraTreesClassifier(random_state=seed)),
                ]
            )
        
        elif self.model_name == "gnb":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", GaussianNB()),
                ]
            )
        
        elif self.model_name == "xgbc":
            self.model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", XGBClassifier(random_state=seed)),
                ]
            )

### **Train**

In [6]:
names = ["lgr", "knc", "gnb", "dtc", "etc", "abc", "gbc", "etcs", "rfc", "xgbc"]

df = pd.read_csv(cfg.train_data_read_path)

# Drop all categorical features
df = df.drop(columns=["id", "product_code", "attribute_0", "attribute_1"])

if cfg.show_info:
    breaker()
    for val in set(df.failure):
        print(f"Class {val} count : {df[df.failure == val].shape[0]}")

X = df.iloc[:, :-1].copy().values
y = df.iloc[:, -1].copy().values

X = si_mean.fit_transform(X)

smote = imblearn.over_sampling.SMOTE(random_state=cfg.seed)
X, y = smote.fit_resample(X, y)

features = [i for i in range(X.shape[1])]

feature_transformer = Pipeline(
    steps=[
        ("Standard_Scaler", StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("features", feature_transformer, features),
    ]
)

best_auc = 0.0
for name in names:
    fold = 1
    if cfg.show_info: breaker()
    for tr_idx, va_idx in KFold(n_splits=cfg.n_splits, random_state=cfg.seed, shuffle=True).split(X):
        X_train, X_valid, y_train, y_valid = X[tr_idx], X[va_idx], y[tr_idx], y[va_idx]
        my_pipeline = Model(name, preprocessor, cfg.seed)
        my_pipeline.model.fit(X_train, y_train)

        y_pred = my_pipeline.model.predict(X_valid)
        if cfg.show_info:
            acc, auc, pre, rec, f1 = get_scores(y_valid, y_pred)
            print(f"{my_pipeline.model_name}, {fold}\n")
            print_scores(acc, auc, pre, rec, f1)
            print("")
        else:
            _, auc, _, _, _ = get_scores(y_valid, y_pred)

        if auc > best_auc:
            best_auc = auc
            model_fold_name = f"{name}_{fold}"
            
            with open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "wb") as fp:
                pickle.dump(my_pipeline.model, fp)
        fold += 1
    

if cfg.show_info: 
    breaker()
    print(f"Best Model : {model_fold_name.split('_')[0]}, Best Fold : {model_fold_name.split('_')[1]}")

breaker()


**************************************************

Class 0 count : 20921
Class 1 count : 5649

**************************************************

lgr, 1

Accuracy  : 0.56865
ROC-AUC   : 0.56872
Precision : [0.61243793 0.52391304]
Recall    : [0.56785793 0.56959034]
F-Score   : [0.58930603 0.54579768]

lgr, 2

Accuracy  : 0.56506
ROC-AUC   : 0.56581
Precision : [0.60611901 0.52465624]
Recall    : [0.55651405 0.57510395]
F-Score   : [0.5802583  0.54872303]

lgr, 3

Accuracy  : 0.56788
ROC-AUC   : 0.56824
Precision : [0.60329986 0.53248925]
Recall    : [0.56316964 0.57330247]
F-Score   : [0.58254445 0.55214268]

lgr, 4

Accuracy  : 0.56477
ROC-AUC   : 0.56660
Precision : [0.63065023 0.50035453]
Recall    : [0.55240313 0.58079561]
F-Score   : [0.58893905 0.53758253]

lgr, 5

Accuracy  : 0.55688
ROC-AUC   : 0.55701
Precision : [0.6063477 0.5065123]
Recall    : [0.55579679 0.55821372]
F-Score   : [0.57997281 0.53110774]


**************************************************

knc, 1

Accurac

### **Predict**

In [7]:
df = pd.read_csv(cfg.test_data_read_path)

# Drop all categorical features
df = df.drop(columns=["id", "product_code", "attribute_0", "attribute_1"])

X = df.copy().values
X = si_mean.transform(X)

model = pickle.load(open(os.path.join(cfg.model_save_path, f"best_model.pkl"), "rb"))
y_pred = model.predict_proba(X)[:, 1]

ss_df = pd.read_csv(cfg.ss_data_read_path)
ss_df["failure"] = y_pred
ss_df.to_csv("submission.csv", index=False)